In [1]:
from utils_filter import *

Subsets outlooks, pph, and report data in various ways (add label to each datapoint for each method of subsetting so can be pulled out of full dataset later)


In [2]:
data_location = 'data'
moderate = False # only consider moderate days
outlooks, pph, reports = read_datasets(data_location, moderate)

reading outlooks 1
reading outlooks 2


c:\Users\miles\OneDrive\Documents\UW\Research\utils_filter.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outlooks = outlooks.append(gp.read_file(data_location + '/outlooks/' + mod_string + '_outlooks_2.shp'))


reading pph
reading storm reports


In [69]:
pph

<xarray.Dataset>
Dimensions:             (time: 14975, x: 93, y: 65)
Coordinates:
  * time                (time) datetime64[ns] 1979-01-01T12:00:00 ... 2019-12...
  * x                   (x) float64 0.0 1.0 2.0 3.0 4.0 ... 89.0 90.0 91.0 92.0
  * y                   (y) float64 0.0 1.0 2.0 3.0 4.0 ... 61.0 62.0 63.0 64.0
Data variables:
    lat                 (y, x) float64 ...
    lon                 (y, x) float64 ...
    p_perfect_wind      (time, y, x) float64 ...
    p_perfect_sig_wind  (time, y, x) float64 ...
    p_perfect_hail      (time, y, x) float64 ...
    p_perfect_sig_hail  (time, y, x) float64 ...
    p_perfect_tor       (time, y, x) float64 ...
    p_perfect_sig_tor   (time, y, x) float64 ...
Attributes:
    title:         Practically Perfect Wind Hindcasts
    sigma:         1.5
    grid:          80-km NCEP 211
    author:        Dr. Victor Gensini
    author_email:  vgensini@niu.edu
    citation:      https://doi.org/10.1175/BAMS-D-19-0321.1

In [62]:
def add_outlooks_label(outlooks, label_dates, labels, label_name, none_label):
    outlooks[label_name] = none_label
    for label, dates in zip(labels, label_dates):
        #print(label)
        outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label
    return outlooks

def add_pph_label(pph, label_dates, labels, label_name, none_label):
    # TODO: add data variable with correct labels with only time as coordinate
    return pph

def add_reports_label(reports, label_dates, labels, label_name, none_label):
    # TODO: verify that date column handles time zones correctly
    # TODO: add column with correct labels
    return reports

def add_labels(outlooks, pph, reports, label_dates, labels, label_name, none_label):
    # adds labels, overwriting with later ones if a date has multiple labels
    return(add_outlooks_label(outlooks, label_dates, labels, label_name, none_label), 
           add_pph_label(pph, label_dates, labels, label_name, none_label),
           add_reports_label(reports, label_dates, labels, label_name, none_label))



Subset by threshold

In [66]:
# add max threshold forecasted for valid day to each datapoint

categories = ['TSTM', 'MRGL', 'SLGT', 'ENH', 'MDT', 'HIGH']
category_dates = []
for category in categories:
    category_dates.append(identify_dates_above_threshold(outlooks, category))

(new_outlooks, new_reports, new_pph) = add_labels(outlooks, pph, reports, category_dates, categories, 'Categorical Risk', 'NONE')

TSTM
MRGL
SLGT
ENH
MDT
HIGH


C:\Users\miles\AppData\Local\Temp\ipykernel_1288\2052472136.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label


Subset by ramp up/down

In [ ]:
# define and add ramp category for each datapoint. Potentially add 2 binary ramp up and ramp down (4 options are [up, down, up and down, niether]). How many forecasts to consider for each day? The day 3, both day 2, and the first day 1 (so 4 forecasts ramp)

Subset by accurate/inaccurate

[array(['201410250000', '201410270000', '201410280000', ...,
        '202312220000', '202312230000', '202312240000'], dtype=object),
 array(['198702140000', '198702270000', '198702280000', ...,
        '202306270000', '202306290000', '202308070000'], dtype=object)]

Consider forecast issue time for day 1--subset reports (and revise pph?)--look at other studies to see how they handled multiple day 1 outlooks (see the one in slack). Starting point should be use first forecasts, later ones would be a different analysis for another time. Decide if we should denote these somehow..

Subset by season

In [ ]:
# add column denoting season (4 met seasons as starting point)

Subset by region

In [ ]:
# decide how to do... lower priotity, could do by state of center of highest category (or center of PPH?) Could start with super basic version

Subset by environmental data (to do later)

In [ ]:
# resave labelled data
# TODO: create master function that subsets data based on easy inputs once it's all been labelled. Put in a util file